# BOT DE ANALITICO DAS PROPOSTAS

In [ ]:
# TRATAMENTO DOS DADOS DO CNES, SIGTAP E TETO
## Autor: Otávio Augusto dos Santos
## Data: 2023-04-13

## Versão: 2.7.10
## Descrição: Bot de analise de dados na planilha de proposto (PLANO)
## Entrada: Planilha de proposto (PLANO)
## Saída: Relatório de inconsistências
## Observações:
## 1. O arquivo de entrada deve estar na pasta PLANILHA 
## 2. O arquivo de saída será gerado na pasta RESULTADOS
## 3. O arquivo de saída será salvo 2 Arquivos: TXT e XLSX

In [ ]:
#Pandas: Possui inúmeras funções e comandos para importar arquivos, analisar dados, tratar dados, etc.
import pandas as pd # importando a biblioteca pandas

#Numpy: Possui uma série de funções e comandos para trabalharmos com números de forma em geral(formatação, calculos, etc)
import numpy as np # importando a biblioteca numpy

#Time: Possui uma série de funções e comandos para trabalharmos com tempo
import time # importando a biblioteca time

#glob: Possui uma série de funções e comandos para trabalharmos com arquivos
import glob # importando a biblioteca glob

#os: Possui uma série de funções e comandos para trabalharmos com arquivos
import os # importando a biblioteca os

#XlsxWriter: Possui uma série de funções e comandos para trabalharmos com arquivos excel
import xlsxwriter # importando a biblioteca xlsxwriter

#Pyexcel: Possui uma série de funções e comandos para trabalharmos com arquivos excel
import pyexcel as pe # importando a biblioteca pyexcel

#locale: Possui uma série de funções e comandos para trabalharmos com formatação de números
import locale # importando a biblioteca locale

#Math: Possui uma série de funções e comandos para trabalharmos com matemática
import math # importando a biblioteca math

#Warnings: Possui detalhes sobre os avisos e alertas que aparecem, porém podemos utiliza-lo também para que os alertas de
#futuras atualizações e metodos depreciados não sejam exibidos
import warnings
warnings.filterwarnings("ignore") 

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8') # Definindo o locale para pt_BR
tempo_inicial = time.time() # tempo inicial para calcular o tempo de execução do código

from glob import glob # Utilizado para listar arquivos de um diretório
from datetime import datetime # Utilizado para trabalhar com datas

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Importação dos Dados do CNES 

In [ ]:
df_cnes_leitos = pd.read_csv('BASE\.BASE_CNES_LEITOS.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_leitos.head()

In [ ]:
df_cnes_habilitacao = pd.read_csv('BASE\.BASE_CNES_HABILITACAO.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_habilitacao.head()

In [ ]:
df_cnes_servicos = pd.read_csv('BASE\.BASE_CNES_SERVICOS.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_servicos.head()

## Importação dos Dados do SIGTAP

In [ ]:
df_sigtap = pd.read_csv('BASE\.BASE_SIGTAP_GERAL.csv', sep=';', encoding='latin-1', dtype=str)
df_sigtap['COD_PROCEDIMENTO']= df_sigtap['COD_PROCEDIMENTO'].astype(int) # Converte a coluna 'COD_PROCEDIMENTO' para string
df_sigtap['QT_PROD AIH 2018 a 2022']= df_sigtap['QT_PROD AIH 2018 a 2022'].astype(int) # Converte a coluna 'QT_PROD AIH 2018 a 2022' para string
df_sigtap.head() # Exibe as 5 primeiras linhas do arquivo

## Importação TETO MAC

In [ ]:
df_teto = pd.read_csv('BASE\.BASE_TETO_MAC.csv', sep=';', encoding='latin-1', dtype=str)
df_teto.head()

## Importação da PLANILHA  

### Analise e tratamento da PLANILHA ABA 1 

In [ ]:
df_planilha = glob('PLANILHA\*.xlsx')[0] # Planilha para ser validada

In [ ]:
df_planilha_aba1 = pd.read_excel(df_planilha, sheet_name='Ident. Fila na UF') # Lê o arquivo excel
df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

In [ ]:
df_planilha_aba1.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA':'COD_PROCEDIMENTO','Unnamed: 1':'DESC_PROCEDIMENTO','Unnamed: 2':'QUANT_FILA', 
                                 'Unnamed: 3':'PERC_REDUCAO', 'Unnamed: 4':'TEMPO_MESES', 'Unnamed: 5':'QUANT_REDUCAO', 'Unnamed: 6':'LINHA'}, inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'
df_planilha_aba1.drop(0, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba1.drop(1, inplace=True) # Remove a segunda linha do arquivo
df_planilha_aba1.drop(df_planilha_aba1.tail(1).index,inplace=True) # Removendo a última linha do arquivo
df_planilha_aba1.dropna(subset=['COD_PROCEDIMENTO'], inplace=True) # Removendo linhas com valores nulos
df_planilha_aba1.drop('Unnamed: 7', axis=1, inplace=True) # Removendo coluna 'Unnamed: 7'
df_planilha_aba1['COD_PROCEDIMENTO'] = df_planilha_aba1['COD_PROCEDIMENTO'].astype(int) # Converte a coluna 'COD_PROCEDIMENTO' para string
df_planilha_aba1['QUANT_FILA'] = df_planilha_aba1['QUANT_FILA'].astype(int) # Converte a coluna 'QUANT_FILA' para string
df_planilha_aba1['PERC_REDUCAO'] = df_planilha_aba1['PERC_REDUCAO'].replace(np.nan, 0) # Substitui os valores nulos por 0
df_planilha_aba1['PERC_REDUCAO'] = df_planilha_aba1['PERC_REDUCAO'].astype(float) # Converte a coluna 'QUANT_FILA' para string
#df_planilha_aba1['QUANT_REDUCAO'] = round(df_planilha_aba1['QUANT_FILA'] * df_planilha_aba1['PERC_REDUCAO']).astype(int) # Converte a coluna 'QUANT_FILA' para string
df_planilha_aba1['QUANT_REDUCAO'] = df_planilha_aba1['QUANT_REDUCAO'] * df_planilha_aba1['PERC_REDUCAO'] # Converte a coluna 'QUANT_FILA' para string
df_planilha_aba1['QUANT_REDUCAO'] = df_planilha_aba1['QUANT_REDUCAO'].apply(math.ceil).astype(int) # Converte a coluna 'QUANT_FILA' para string

df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

In [ ]:
df_planilha_aba1['PERC_REDUCAO'] = df_planilha_aba1['PERC_REDUCAO'].replace(np.nan, 0) # Substitui os valores nulos por 0 
df_planilha_aba1.info() # Exibe informações sobre o arquivo

### Somas TOTAIS

In [ ]:
quant_fila = df_planilha_aba1['QUANT_FILA'].sum() # Soma o valor total da coluna 'QUANT_FILA'
quant_fila = '{0:,}'.format(quant_fila).replace(',','.') #Aqui coloca os pontos
quant_reducao = df_planilha_aba1['QUANT_REDUCAO'].sum() # Soma o valor total da coluna 'QUANT_REDUCAO'
quant_reducao = '{0:,}'.format(quant_reducao).replace(',','.') #Aqui coloca os pontos
quant_prodedimentos = df_planilha_aba1['COD_PROCEDIMENTO'].count() # Conta a quantidade de procedimentos
reducao_max = df_planilha_aba1['PERC_REDUCAO'].max() # Pega o valor máximo da coluna 'PERC_REDUCAO'
reducao_max = "{:.0f}".format(reducao_max * 100)  # Formata o valor para 2 casas decimais
reducao_min = df_planilha_aba1.loc[df_planilha_aba1['PERC_REDUCAO'] > 0, 'PERC_REDUCAO'].min()
reducao_min = "{:.0f}".format(reducao_min * 100)  # Formata o valor para 2 casas decimais
periodo_atendimento = df_planilha_aba1.loc[df_planilha_aba1['QUANT_REDUCAO'] > 0, 'TEMPO_MESES'].max()
#periodo_atedimento = 'erro'

### Periodo de atendimento

In [ ]:
df_planilha_aba1['TEMPO_MESES'] = np.where(df_planilha_aba1['PERC_REDUCAO'] > 0, df_planilha_aba1['TEMPO_MESES'], '-')
df_planilha_aba1.head()

### Verificar procedimento duplicado

In [ ]:
# Verifica se há valores duplicados na coluna "COD_PROCEDIMENTO"
df_planilha_aba1['DUPLICADO'] = df_planilha_aba1['COD_PROCEDIMENTO'].duplicated(keep=False)

# Cria uma nova coluna com as mensagens correspondentes
df_planilha_aba1['M_DUPLICADO'] = np.where(df_planilha_aba1['DUPLICADO'], 'SIM', '-')
df_planilha_aba1.drop('DUPLICADO', axis=1, inplace=True) # Remove a coluna 'DUPLICADO'
# Exibe as 5 primeiras linhas do arquivo com a nova coluna
df_planilha_aba1.head()

### Verificar se existe atendimento na Quantidade de Reduçao numero inferior a 1 procedimento. 

In [ ]:
# Adicionando nova coluna 'QUANT_ZERO' com a mensagem correspondente

def mensagem(quant_zero):
    if quant_zero ['QUANT_REDUCAO'] < 1 and quant_zero['PERC_REDUCAO'] > 0 :
        return 'ERRO_QUANT'
    else:
        return '-'
df_planilha_aba1['QUANT_ZERO'] = df_planilha_aba1.apply(mensagem, axis=1)

# Exibindo as primeiras linhas do dataframe com a nova coluna
df_planilha_aba1.head() 


### Procedimento requer habilitação

In [ ]:
df_sigtap_h = df_sigtap[['COD_PROCEDIMENTO','EXIGE HABILITACAO','CO_HABILITACAO']] # Cria um novo dataframe com as colunas 'COD_PROCEDIMENTO','EXIGE HABILITACAO','CO_HABILITACAO'
df_sigtap_h.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'COD_PROCEDIMENTO'
df_planilha_aba1['PROC_HABILITACAO'] = df_planilha_aba1['COD_PROCEDIMENTO'].map(df_sigtap_h.set_index('COD_PROCEDIMENTO')['EXIGE HABILITACAO']) # Adiciona uma nova coluna com a informação de habilitação do procedimento
df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

### Procedimento requer serviço/class

In [ ]:
df_sigtap_s = df_sigtap[['COD_PROCEDIMENTO','EXIGE SERVIÇO','CO_SERVICO','CO_CLASSIFICACAO']] # Cria um novo dataframe com as colunas 'COD_PROCEDIMENTO','EXIGE SERVICO','CO_SERVICO'
df_sigtap_s.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'COD_PROCEDIMENTO'
df_planilha_aba1['PROC_SERVICO'] = df_planilha_aba1['COD_PROCEDIMENTO'].map(df_sigtap_s.set_index('COD_PROCEDIMENTO')['EXIGE SERVIÇO']) # Adiciona uma nova coluna com a informação de serviço do procedimento
df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

## Analise e tratamento da PLANILHA ABA 2

In [ ]:
df_planilha_aba2 = pd.read_excel(df_planilha, sheet_name='Ident. CNES e Proced.', dtype=str) # Lê o arquivo excel
df_planilha_aba2.head() # Exibe as 5 primeiras linhas do arquivo

In [ ]:
# Tratamento dos dados da aba 'Ident. CNES e Proced.'
df_planilha_aba2.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - CNES':'CNES','Unnamed: 1':'ESTABELECIMENTO','Unnamed: 2':'COD_PROCEDIMENTO', 
                                 'Unnamed: 3':'DES_PROCEDIMENTO', 'Unnamed: 4':'COMPL_RECURSO_FEDERAL', 'Unnamed: 5':'GESTAO', 'Unnamed: 6':'COD_NATUREZA','Unnamed: 7':'NATUREZA',
                                 'Unnamed: 8':'POSSUI_CONTRATO','Unnamed: 9':'IDENTIFICACAO','Unnamed: 10':'LINHA'}, inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'
df_planilha_aba2.drop(0, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba2.drop(1, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba2.drop(df_planilha_aba2.tail(1).index,inplace=True) # Removendo a última linha do arquivo
df_planilha_aba2.dropna(subset=['CNES'], inplace=True) # Removendo linhas com valores nulos
df_planilha_aba2.drop('IDENTIFICACAO', axis=1, inplace=True) # Removendo coluna 'IDENTIFICACAO'
df_planilha_aba2.dropna(subset=['COD_PROCEDIMENTO'], inplace=True) # Removendo linhas com valores nulos
df_planilha_aba2['COD_PROCEDIMENTO'] = df_planilha_aba2['COD_PROCEDIMENTO'].astype(int) # Converte a coluna 'COD_PROCEDIMENTO' para string
df_cnes_habilitacao = df_cnes_habilitacao.rename(columns={"CO_CNES": "CNES"}) # Renomeia a coluna 'CO_CNES' para 'CNES'
df_planilha_aba2 = df_planilha_aba2.merge(df_cnes_habilitacao[["CNES", "CO_MUNICIPIO_GESTOR"]], on="CNES", how="left") # Adiciona a coluna 'CO_MUNICIPIO_GESTOR' ao dataframe

df_planilha_aba2.head() # Exibe as 5 primeiras linhas do arquivo

In [ ]:
df_planilha_aba2.info() 

### Verificar se o CNES esta ATIVO:

In [ ]:
df_cnes_habilitacao['CNES'] = df_cnes_habilitacao['CNES'].astype(str) # Converte a coluna 'CNES' para string
df_cnes_habilitacao2 = df_cnes_habilitacao.loc[df_cnes_habilitacao['CO_MOTIVO_DESAB'] > '0'] # Seleciona apenas os CNES habilitados
df_planilha_aba2['CNES_ATIVO'] = np.where(df_planilha_aba2['CNES'].isin(df_cnes_habilitacao2['CNES']), 'NÃO', '-') # Adiciona a coluna 'CNES_ATIVO' ao dataframe
df_planilha_aba2.head() # Exibe as 5 primeiras linhas do arquivo

### Verificar se existe prestador sem procedimento informado em fila (relação ABA1 E ABA2)

In [ ]:
# Idenfica se o procedimento informado na aba anterior está na aba atual
df_planilha_aba2['PROC_INFORMADO'] = np.where(df_planilha_aba2['COD_PROCEDIMENTO'].isin(df_planilha_aba1['COD_PROCEDIMENTO']), '-','NÃO')     
df_planilha_aba2.head()                                

### Verificar se existe procedimento sem prestador informado (relação ABA2 e ABA1)

In [ ]:
# Verifica se o procedimento informado possui prestador informado
df_planilha_aba1_p = df_planilha_aba1.loc[df_planilha_aba1['QUANT_REDUCAO'] > 0] # Seleciona apenas os procedimentos com quantidade de redução maior que zero
df_planilha_aba1_p['POSSUI_PRESTADOR'] = np.where(df_planilha_aba1_p['COD_PROCEDIMENTO'].isin(df_planilha_aba2['COD_PROCEDIMENTO']),'-','NÃO') 
df_planilha_aba1 = df_planilha_aba1.merge(df_planilha_aba1_p[['COD_PROCEDIMENTO','POSSUI_PRESTADOR']], on='COD_PROCEDIMENTO', how='left') # Adiciona a coluna 'POSSUI_PRESTADOR' ao dataframe
df_planilha_aba1.head()

### Verificar se o procedimento informado é valido 

In [ ]:
df_planilha_aba1['PROC_VALIDO'] = np.where(df_planilha_aba1['COD_PROCEDIMENTO'].isin(df_sigtap['COD_PROCEDIMENTO']), '-','NÃO')
df_planilha_aba1.head()

### Verificar habilitação x CNES

In [ ]:
df_planilha_aba2_h = df_planilha_aba2[['CNES','COD_PROCEDIMENTO']] # Cria um novo dataframe com as colunas 'CNES','COD_PROCEDIMENTO',
df_planilha_aba2_h.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'COD_PROCEDIMENTO'
df_planilha_aba2_h['PROC_HABILITACAO'] = df_planilha_aba2_h['COD_PROCEDIMENTO'].map(df_sigtap_h.set_index('COD_PROCEDIMENTO')['EXIGE HABILITACAO']) # Adiciona uma nova coluna com a informação de habilitação do procedimento
df_planilha_aba2_h = df_planilha_aba2_h.merge(df_sigtap_h[['COD_PROCEDIMENTO','CO_HABILITACAO']], on='COD_PROCEDIMENTO', how='left') # Adiciona a coluna 'PROC_VALIDO' ao dataframe
df_planilha_aba2_h = df_planilha_aba2_h.merge(df_cnes_habilitacao[['CNES','CO_CODIGO_GRUPO']], on='CNES', how='left') # Adiciona a coluna 'PROC_VALIDO' ao dataframe
df_planilha_aba2_h.drop(df_planilha_aba2_h.loc[df_planilha_aba2_h['PROC_HABILITACAO'] == '-'].index, inplace=True) # Remove os procedimentos que não exigem habilitação
df_planilha_aba2_h['CNES_HABILITADO'] = np.where(df_planilha_aba2_h['CO_CODIGO_GRUPO'].isin(df_planilha_aba2_h['CO_HABILITACAO']), 'SIM','EXIGE_HAB') # Adiciona a coluna 'CNES_HABILITADO' ao dataframe
df_planilha_aba2_h.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'CNES'
df_planilha_aba2 = df_planilha_aba2.merge(df_planilha_aba2_h[['CNES','COD_PROCEDIMENTO','CNES_HABILITADO']], on=['CNES','COD_PROCEDIMENTO'], how='left') # Adiciona a coluna 'CNES_HABILITADO' ao dataframe
df_planilha_aba2.drop_duplicates(subset='LINHA', keep='first', inplace=True) # Remove os valores duplicados da coluna 'CNES'
df_planilha_aba2.head()

### Verificar serviços/class x CNES

In [ ]:
df_planilha_aba2_s = df_planilha_aba2[['CNES','COD_PROCEDIMENTO']] # Cria um novo dataframe com as colunas 'CNES','COD_PROCEDIMENTO',
df_planilha_aba2_s.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'COD_PROCEDIMENTO'
df_planilha_aba2_s['EXIGE SERVIÇO'] = df_planilha_aba2_s['COD_PROCEDIMENTO'].map(df_sigtap_s.set_index('COD_PROCEDIMENTO')['EXIGE SERVIÇO']) # Adiciona uma nova coluna com a informação de habilitação do procedimento
df_planilha_aba2_s = df_planilha_aba2_s.merge(df_sigtap_s[['COD_PROCEDIMENTO','CO_SERVICO']], on='COD_PROCEDIMENTO', how='left') # Adiciona a coluna 'PROC_VALIDO' ao dataframe
df_cnes_servicos = df_cnes_servicos.rename(columns={"CO_CNES": "CNES"}) # Renomeia a coluna 'CO_CNES' para 'CNES'
df_planilha_aba2_s = df_planilha_aba2_s.merge(df_cnes_servicos[['CNES','CO_SERVICO']], on='CNES', how='left') # Adiciona a coluna 'PROC_VALIDO' ao dataframe
df_planilha_aba2_s.drop(df_planilha_aba2_s.loc[df_planilha_aba2_s['EXIGE SERVIÇO'] == '-'].index, inplace=True) # Remove os procedimentos que não exigem habilitação
df_planilha_aba2_s['CNES_SERVICO'] = np.where(df_planilha_aba2_s['CO_SERVICO_x'].isin(df_planilha_aba2_s['CO_SERVICO_y']), '-','EXIGE_SERV') # Adiciona a coluna 'CNES_HABILITADO' ao dataframe
df_planilha_aba2_s.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'CNES'
df_planilha_aba2 = df_planilha_aba2.merge(df_planilha_aba2_s[['CNES','COD_PROCEDIMENTO','CNES_SERVICO']], on=['CNES','COD_PROCEDIMENTO'], how='left') # Adiciona a coluna 'CNES_HABILITADO' ao dataframe
df_planilha_aba2.drop_duplicates(subset='LINHA', keep='first', inplace=True) # Remove os valores duplicados da coluna 'CNES'
df_planilha_aba2.head()

### Verificar tipo de Gestão

In [ ]:
df_planilha_aba2_g = df_planilha_aba2[['CNES','GESTAO','LINHA']] # Cria um novo dataframe com as colunas 'CNES','COD_PROCEDIMENTO', 
df_cnes_gestao = df_cnes_leitos[['CO_CNES','TP_GESTAO']] # Cria um novo dataframe com as colunas 'CNES','COD_PROCEDIMENTO',]
df_cnes_gestao = df_cnes_gestao.rename(columns={"CO_CNES": "CNES"}) # Renomeia a coluna 'CO_CNES' para 'CNES'
df_planilha_aba2_g = df_planilha_aba2_g.merge(df_cnes_gestao[['CNES','TP_GESTAO']], on='CNES', how='left') # Adiciona a coluna 'PROC_VALIDO' ao dataframe
df_planilha_aba2_g['TP_GESTAO'] = df_planilha_aba2_g['TP_GESTAO'].replace({'M': 'MUNICIPAL', 'E': 'ESTADUAL', 'D': 'DUPLA'})
df_planilha_aba2_g.drop_duplicates(subset='LINHA', keep='first', inplace=True) # Remove os valores duplicados da coluna 'COD_PROCEDIMENTO'

df_planilha_aba2_g['GESTAO_VALIDA'] =   np.where((df_planilha_aba2_g['GESTAO'] == 'MUNICIPAL') & (df_planilha_aba2_g['TP_GESTAO'] == 'MUNICIPAL'), '-', 
                                        np.where((df_planilha_aba2_g['GESTAO'] == 'ESTADUAL') & (df_planilha_aba2_g['TP_GESTAO'] == 'ESTADUAL'), '-', 
                                        np.where((df_planilha_aba2_g['GESTAO'] == 'MUNICIPAL') & (df_planilha_aba2_g['TP_GESTAO'] == 'DUPLA'), '-', 
                                        np.where((df_planilha_aba2_g['GESTAO'] == 'ESTADUAL') & (df_planilha_aba2_g['TP_GESTAO'] == 'DUPLA'), '-', 
                                        np.where((df_planilha_aba2_g['GESTAO'] == 'MUNICIPAL') & (df_planilha_aba2_g['TP_GESTAO'] == 'ESTADUAL'), 'NÃO', 
                                        np.where((df_planilha_aba2_g['GESTAO'] == 'ESTADUAL') & (df_planilha_aba2_g['TP_GESTAO'] == 'MUNICIPAL'), 'NÃO', '-')))))) # verificar gestão

df_planilha_aba2 = df_planilha_aba2.merge(df_planilha_aba2_g[['LINHA','GESTAO_VALIDA']], on='LINHA', how='left') # Adiciona a coluna 'CNES_HABILITADO' ao dataframe
df_planilha_aba2.head()

### Somas TOTAIS

In [ ]:
quant_cnes = df_planilha_aba2['CNES'].nunique() # Quantidade de CNES
quant_cnes_municipal = df_planilha_aba2['CNES'].loc[df_planilha_aba2['GESTAO'] == 'MUNICIPAL'].nunique() # Quantidade de municípios
quant_cnes_estadual = df_planilha_aba2['CNES'].loc[df_planilha_aba2['GESTAO'] == 'ESTADUAL'].nunique() # Quantidade de estadual

## Analise e tratamento da PLANILHA ABA 3

In [ ]:
df_planilha_aba3 = pd.read_excel(df_planilha, sheet_name='Execução') # Lê o arquivo excel
df_planilha_aba3.head(10) # Exibe as 5 primeiras linhas do arquivo

In [ ]:
valor_portaria = df_planilha_aba3.iloc[0,3] # Armazena o valor da portaria
df_planilha_aba3.drop(df_planilha_aba3.index[0:6], inplace=True) # Remove as 5 primeiras linhas do arquivo
df_planilha_aba3.rename(columns={'Distribuição e Cronograma da Execução do Recurso Financeiro':'CODIGO GESTOR','Unnamed: 1':'GESTÃO','Unnamed: 2':'DESC_GESTOR','Unnamed: 3':'VALOR','Unnamed: 4':'MARÇO',
                                 'Unnamed: 5':'ABRIL','Unnamed: 6':'MAIO','Unnamed: 7':'JUNHO','Unnamed: 8':'JULHO','Unnamed: 9':'AGOSTO','Unnamed: 10':'SETEMBRO','Unnamed: 11':'OUTUBRO',
                                 'Unnamed: 12':'NOVEMBRO','Unnamed: 13':'DEZEMBRO','Unnamed: 14':'TOTAL_%','SQ (CODIGO Interno':'LINHA'}, inplace=True)
df_planilha_aba3.drop(df_planilha_aba3.tail(1).index,inplace=True) # Removendo a última linha do arquivo
df_planilha_aba3.dropna(subset=['CODIGO GESTOR'], inplace=True) # Removendo linhas com valores nulos
valor_total = df_planilha_aba3['VALOR'].sum() # Soma o valor total da coluna 'VALOR'
df_planilha_aba3['VALOR'] = df_planilha_aba3['VALOR'].astype(float) # Converte a coluna 'VALOR' para float
df_planilha_aba3['VALOR_R'] = df_planilha_aba3['VALOR'].apply(lambda x: 'R$ ' + format(x, ',.2f').replace('.', '#').replace(',', '.').replace('#', ',')) # Formata a coluna 'VALOR' para moeda

df_planilha_aba3.head() # Exibe as 5 primeiras linhas do arquivo

In [ ]:
df_planilha_aba3['CODIGO GESTOR'] = df_planilha_aba3['CODIGO GESTOR'].astype(int) # Converte a coluna 'MUNIC_PRESTADOR' para string
df_planilha_aba2['CO_MUNICIPIO_GESTOR'] = df_planilha_aba2['CO_MUNICIPIO_GESTOR'].replace(np.nan, 0) # Substitui os valores nulos por 0
df_planilha_aba2['CO_MUNICIPIO_GESTOR'] = df_planilha_aba2['CO_MUNICIPIO_GESTOR'].astype(int) # Converte a coluna 'CO_MUNICIPIO_GESTOR' para string
df_planilha_aba3['MUNIC_PRESTADOR'] = np.where(df_planilha_aba3['CODIGO GESTOR'].isin(df_planilha_aba2['CO_MUNICIPIO_GESTOR']), 'SIM','NÃO')
df_planilha_aba3_m = df_planilha_aba3[df_planilha_aba3['GESTÃO'] == 'MUNICIPAL'] # Filtra os dados da aba 3 que possuem gestão municipal
df_teto['Código IBGE'] = df_teto['Código IBGE'].astype(int) # Converte a coluna 'Código IBGE' para string
df_planilha_aba3 = pd.merge(df_planilha_aba3, df_teto, left_on='CODIGO GESTOR', right_on='Código IBGE', how='left') # Realiza o merge das duas planilhas


df_planilha_aba2_g['TP_GESTAO'] = df_planilha_aba2_g['TP_GESTAO'].replace({'M': 'MUNICIPAL', 'E': 'ESTADUAL', 'D': 'DUPLA'})

In [ ]:
df_planilha_aba3.drop(['Código IBGE', 'Código Gestão', 'Descrição Gestão'], axis=1, inplace=True) # Remove a coluna 'Código IBGE'
df_planilha_aba3.rename(columns={'Descrição Gestão':'DESC_GESTOR','Estado / Município':'DESC_GESTOR_C','Teto Financeiro MAC - Valores Anuais (R$)':'TETO_FINANC_ANO'}, inplace=True) # Renomeia a coluna 'Descrição Gestão'
df_planilha_aba3['TETO_FINANC_ANO'] = df_planilha_aba3['TETO_FINANC_ANO'].astype(float) # Converte a coluna 'TETO_FINANC_ANO' para float
df_planilha_aba3['TETO_FINANC_MES'] = df_planilha_aba3['TETO_FINANC_ANO'] / 12 # Calcula o teto financeiro mensal
df_planilha_aba3.dropna(subset=['DESC_GESTOR'], inplace=True) # Remove as linhas com valores nulos

df_planilha_aba3.head()

In [ ]:
df_planilha_aba3['VALOR'] = df_planilha_aba3['VALOR'].astype(float) # Converte a coluna 'VALOR' para float
df_planilha_aba3['VALOR'] = df_planilha_aba3['VALOR'].apply(lambda x: 'R$ ' + format(x, ',.2f').replace('.', '#').replace(',', '.').replace('#', ',')) # Formata a coluna 'VALOR' para moeda
df_planilha_aba3['TETO_FINANC_ANO'] = df_planilha_aba3['TETO_FINANC_ANO'].astype(float) # Converte a coluna 'TETO_FINANC_MES' para float
df_planilha_aba3['TETO_FINANC_ANO'] = df_planilha_aba3['TETO_FINANC_ANO'].apply(lambda x: 'R$ ' + format(x, ',.2f').replace('.', '#').replace(',', '.').replace('#', ',')) # Formata a coluna 'TETO_FINANC_MES' para moeda
df_planilha_aba3['TETO_FINANC_MES'] = df_planilha_aba3['TETO_FINANC_MES'].astype(float) # Converte a coluna 'TETO_FINANC_MES' para float
df_planilha_aba3['TETO_FINANC_MES'] = df_planilha_aba3['TETO_FINANC_MES'].apply(lambda x: 'R$ ' + format(x, ',.2f').replace('.', '#').replace(',', '.').replace('#', ',')) # Formata a coluna 'TETO_FINANC_MES' para moeda

cols = ['MARÇO','ABRIL','MAIO','JUNHO','JULHO','AGOSTO','SETEMBRO','OUTUBRO','NOVEMBRO','DEZEMBRO','TOTAL_%'] # Lista de colunas
df_planilha_aba3[cols] = df_planilha_aba3[cols].fillna(0).astype(float) # Converte os valores para float
df_planilha_aba3[cols] = df_planilha_aba3[cols].applymap(lambda x: '{:.0%}'.format(float(x))) # Formata as colunas para porcentagem


df_planilha_aba3.head() 

### Verificar se a CNES de gestão Municipal esta contemplado na Execução

In [ ]:
df_planilha_aba2_e = df_planilha_aba2[['CNES','GESTAO','CO_MUNICIPIO_GESTOR']] # Filtra as colunas 'CNES' e 'GESTAO' da aba 2
df_planilha_aba2_e.drop(df_planilha_aba2_e.loc[df_planilha_aba2_e['GESTAO']!='MUNICIPAL'].index, inplace=True) # Remove as linhas que possuem gestão municipal
df_planilha_aba2_e = df_planilha_aba2_e.drop_duplicates() # Remove as linhas duplicadas
df_planilha_aba2_e['CNES GESTÃO MUNIC_X EXECUÇÃO'] = np.where(df_planilha_aba2_e['CO_MUNICIPIO_GESTOR'].isin(df_planilha_aba3['CODIGO GESTOR']), '-','NÃO') # Verifica se o CNES da aba 2 está na aba 3
df_planilha_aba2 = df_planilha_aba2.merge(df_planilha_aba2_e[['CNES','CNES GESTÃO MUNIC_X EXECUÇÃO']], on='CNES', how='left') # Realiza o merge das duas planilhas
df_planilha_aba2.head ()

### Somas TOTAIS

In [ ]:
valor_portaria = round(valor_portaria, 2) # Arredonda o valor da portaria para duas casas decimais
valor_total = round(valor_total, 2) # Arredonda o valor total para duas casas decimais

diferenca = valor_portaria - valor_total # Calcula a diferença entre o valor da portaria e o valor total
valor_total = 'R$ ' + format(valor_total, ',.2f').replace('.', '#').replace(',', '.').replace('#', ',') # Formata o valor total para moeda

uf = df_planilha_aba3.loc[df_planilha_aba3['GESTÃO'] == 'ESTADUAL', 'DESC_GESTOR'].iloc[0].upper()

# RELATORIO FINAL (ANALISE)

In [ ]:
caminho_nova_pasta = "RESULTADOS"

try:
    os.mkdir(caminho_nova_pasta)
    print("Pasta criada com sucesso!")
except OSError as erro:
    print(f"Não foi possível criar a pasta: {erro}")

In [ ]:
df_planilha = os.path.splitext(os.path.basename(df_planilha))[0] # Pega o nome do arquivo sem a extensão
file_nome = df_planilha.split('/')[-1] # Pega o nome do arquivo
writer = pd.ExcelWriter(f'RESULTADOS/{file_nome}_resultado.xlsx', engine='xlsxwriter') # Cria um arquivo excel

df_planilha_aba1.to_excel(writer, sheet_name='Aba 1', index=False)
df_planilha_aba2.to_excel(writer, sheet_name='Aba 2', index=False)
df_planilha_aba3.to_excel(writer, sheet_name='Aba 3', index=False)

writer.save()
writer.close()


## Tempo de execução

In [ ]:
tempo_final = time.time()
tempo_total = int(tempo_final - tempo_inicial)

minutos = tempo_total // 60
segundos = tempo_total % 60

data_hora_atual = datetime.now() # Pega a data e hora atual

print(f"Tempo total de execução: {minutos} minutos e {segundos} segundos.")

# SALVANDO OS RESULTADOS    

In [ ]:
#Criar arquivo txt resultado em modo de escrita
arquivo = open(f'RESULTADOS/{file_nome}_resultado.txt', 'w')




# Informações do arquivo
print(f"\n=============================================== INFORMAÇÕES DO ARQUIVO ================================================", file=arquivo)

print(f"\n======================================================[ ABA 1 ]========================================================", file=arquivo)

if quant_reducao == '0':
    print(f" [ERRO] - ABA 1 - NÃO existem reduções de procedimentos, campos zerado; ============> SOMA DA COLUNA [QUANT_REDUCAO](F)", file=arquivo)
else:
    print(f" [OK] - ABA 1 - Existem reduções de prodimentos conforme programado; ===============> SOMA DA COLUNA [QUANT_REDUCAO](F)", file=arquivo)

  
# Verificação de procedimentos duplicados
if df_planilha_aba1['M_DUPLICADO'].str.contains('SIM').any():
    print(f" [ERRO] - ABA 1 - Existem procedimentos na Fila, duplicado; ==========================> NOME DA COLUNA [M_DUPLICADO](H)", file=arquivo)
else:
    print(f" [OK] - ABA 1 - Não existem procedimentos duplicados; ================================> NOME DA COLUNA [M_DUPLICADO](H)", file=arquivo)

# VerificaçãO de procedimentos com quantidade menor que 1
if df_planilha_aba1['QUANT_ZERO'].str.contains('ERRO_QUANT').any():   
    print(f" [ERRO] - ABA 1 - Existem procedimentos na Fila,cuja quantidade de cirurgia ficou inferior a 1 PACIENTE;['QUANT_ZERO'](I)", file=arquivo)
else:
    print(f" [OK] - ABA 1 - Não existem procedimentos com quantidade menor que 1; ===============> NOME DA COLUNA ['QUANT_ZERO'](I)", file=arquivo)

# Verificar porcentagem  de procedimentos com quantidade 
if df_planilha_aba1['PERC_REDUCAO'].apply(lambda x: x > 1).any():
    print(f" [ERRO] - ABA 1 - Existem procedimentos na Fila com mais de 100% de redução; ====> VERIFICAR COLUNA ['PERC_REDUCAO'](D)", file=arquivo)
else:
    print(f" [OK] - ABA 1 - Não existem procedimentos na Fila com mais de 100% de redução; ==> VERIFICAR COLUNA ['PERC_REDUCAO'](D)", file=arquivo)

periodo_atedimento = df_planilha_aba1.loc[df_planilha_aba1['QUANT_REDUCAO'] > 0, 'TEMPO_MESES'].max()

# Verificar prazo de atendimento
if df_planilha_aba1['TEMPO_MESES'].apply(pd.isna).any():
#  df_planilha_aba1['QUANT_REDUCAO'] > 0 and df_planilha_aba1['TEMPO_MESES'].apply(pd.isna).any():   
 #                                        df_planilha_aba1['TEMPO_MESES'].apply(pd.isna).any():
    print(f" [ERRO] - ABA 1 - Existem procedimentos na Fila sem prazo de atendimento; ========> VERIFICAR COLUNA ['TEMPO_MESES'](E)", file=arquivo)
else:
    print(f" [OK] - ABA 1 - Não existem procedimentos na Fila sem prazo de atendimento; ======> VERIFICAR COLUNA ['TEMPO_MESES'](E)", file=arquivo)

# Verificação de procedimentos inválidos
if df_planilha_aba1['PROC_VALIDO'].str.contains('NÃO').any():
    print(f" [ERRO] - ABA 1 - Existem procedimentos na Fila, que não são válidos; ==============> NOME DA COLUNA ['PROC_VALIDO'](M)", file=arquivo)
else:
    print(f" [OK] - ABA 1 - Não existem procedimentos inválidos; ===============================> NOME DA COLUNA ['PROC_VALIDO'](M)", file=arquivo)

# Verificação de procedimentos sem prestador
if df_planilha_aba1['POSSUI_PRESTADOR'].str.contains('NÃO').any():
    print(f" [ERRO] - ABA 1/2 - Existem procedimentos na Fila, cuja não existe prestador; => NOME DA COLUNA ['POSSUI_PRESTADOR'](L)", file=arquivo)
else:
    print(f" [OK] - ABA 1/2 - Não existem procedimentos sem prestador; ====================> NOME DA COLUNA ['POSSUI_PRESTADOR'](L)", file=arquivo)



print(f"\n\n======================================================[ ABA 2 ]========================================================", file=arquivo)

# Verificação de prestadores sem procedimento
if df_planilha_aba2['PROC_INFORMADO'].str.contains('NÃO').any():
    print(f" [ERRO] - ABA 2/1 - Existem prestador na Fila, que não possuem procedimentos; ===> NOME DA COLUNA ['PROC_INFORMADO'](M)", file=arquivo)
else:
    print(f" [OK] - ABA 2/1 - Não existem prestadores sem procedimentos; ====================> NOME DA COLUNA ['PROC_INFORMADO'](M)", file=arquivo)

# Verificação de CNES ativo
if df_planilha_aba2['CNES_ATIVO'].str.contains('NÃO').any():
    print(f" [ERRO] - ABA 2 - Existem CNES inativos; ============================================> NOME DA COLUNA ['CNES_ATIVO'](L)", file=arquivo)
else:
    print(f" [OK] - ABA 2 - Não existem CNES inativos; ==========================================> NOME DA COLUNA ['CNES_ATIVO'](L)", file=arquivo)

# Verificação de CNES habilitado
if df_planilha_aba2['CNES_HABILITADO'].str.contains('EXIGE_HAB').any():
    print(f" [ALERTA] - ABA 2 - Existem CNES não habilitados; ==============================> NOME DA COLUNA ['CNES_HABILITADO'](N)", file=arquivo)
else:
    print(f" [OK] - ABA 2 - Não existem CNES não habilitados; ==============================> NOME DA COLUNA ['CNES_HABILITADO'](N)", file=arquivo)

# Verificação de CNES serviço ativo
if df_planilha_aba2['CNES_SERVICO'].str.contains('EXIGE_SERV').any():
    print(f" [ALERTA] - ABA 2 - Existem CNES não serviço/class;==================================> NOME DA COLUNA [CNES_SERVICO](O)", file=arquivo)
else:
    print(f" [OK] - ABA 2 - Não existem CNES não serviço/class;==================================> NOME DA COLUNA [CNES_SERVICO](O)", file=arquivo)

# CNES GESTÃO MUNIC_X EXECUÇÃO
if df_planilha_aba2['CNES GESTÃO MUNIC_X EXECUÇÃO'].str.contains('NÃO').any():
    print(f" [ERRO] - ABA 2 - Existem CNES de gestão municipal não relacionado na ABA 3; =====> ['CNES GESTÃO MUNIC_X EXECUÇÃO'](Q)", file=arquivo)
else:
    print(f" [OK] - ABA 2 - CNES de gestão municipal relacionado na ABA3; =====> NOME DA COLUNA ['CNES GESTÃO MUNIC_X EXECUÇÃO'](Q)", file=arquivo)

# CNES GESTÃO ESTAD_X EXECUÇÃO
if df_planilha_aba2['GESTAO_VALIDA'].str.contains('NÃO').any():
    print(f" [ALERTA] - ABA 2 - Existem CNES informado com gestão diferente do CNES-WEB; =====> NOME DA COLUNA ['GESTAO_VALIDA'](R)", file=arquivo)
else:
    print(f" [OK] - ABA 2 - CNES informado com gestão igual ao CNES-WEB; =====================> NOME DA COLUNA ['GESTAO_VALIDA'](R)", file=arquivo)

# CONTRATO/SERVIÇO SUS
#if df_planilha_aba2['CONTRATO SUS'].str.contains('NÃO').any():
 #   print(f" [ALERTA] - ABA 2 - Existem CNES sem contrato/serviço SUS;", file=arquivo)
#else:
#    print(f" [OK] - ABA 2 - Não existem CNES sem contrato/serviço SUS;", file=arquivo)



print(f"\n\n======================================================[ ABA 3 ]========================================================", file=arquivo)

# Verificar se existe município com repasse, mas não existe CNES relacionado ao municipio
if df_planilha_aba3_m['MUNIC_PRESTADOR'].str.contains('NÃO').any():   
    print(f" [ERRO] - ABA 3 - Existem municipio com repasse, mas não existe CNES relacionado ao município;=> ['MUNIC_PRESTADOR'](R)", file=arquivo)
else:   
    print(f" [OK] - ABA 3 - Relação de município desacordo com o CNES; =====================> NOME DA COLUNA ['MUNIC_PRESTADOR'](R)", file=arquivo)

# Verificar valor de portaria
if diferenca == 0:
    print(f" [OK] - ABA 3 - Valor programado igual ao valor alocado na Portaria 90; ====================> SOMA DA COLUNA [VALOR](D)", file=arquivo)
else:   
    print(f" [ERRO] - ABA 3 - Valor programado diferente do valor alocado na Portaria 90; (Valor diferente: {locale.currency(diferenca, grouping=True)})", file=arquivo)



print(f"\n\n=====================================================[ ARQUIVO ]=======================================================", file=arquivo)

print(f" [OK] - Arquivo enviado pelo gestor: '{file_nome}';", file=arquivo)
print(f" [OK] - Arquivo TXT: '{file_nome} - resultado.txt'  gerado com sucesso;", file=arquivo)
print(f" [OK] - Arquivo XLS: '{file_nome} - resultado.xlsx' gerado com sucesso;", file=arquivo)

# RESULTADO FINAL
print(f"\n \n=================================================== RESULTADO FINAL ===================================================  \n", file=arquivo)
print(f" UF DO PLANO DE AÇÃO ===========================================> {uf}", file=arquivo)
print(f" QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22 ===========> {quant_fila}", file=arquivo)
print(f" QTDE DE CIRURGIAS A SEREM FEITAS NO PRAZO PACTUADO  ===========> {quant_reducao}", file=arquivo)
print(f" QTDE PROCEDIMENTO CIRURGICOS INFORMADO NA FILA   ==============> {quant_prodedimentos}", file=arquivo)
print(f" REDUÇÃO DO TAMANHO DA FILA (%) - MAX e MIN ====================> {reducao_max}% e {reducao_min}%", file=arquivo)
print(f" PRAZO PARA OS ATENDIMENTOS ====================================> {periodo_atendimento} meses", file=arquivo)
print(f" TOTAL DE ESTABELECIMENTOS CNES ================================> {quant_cnes}", file=arquivo)
print(f" TOTAL DE ESTABELECIMENTOS EM GESTÃO MUNICIPAL =================> {quant_cnes_municipal}", file=arquivo)
print(f" TOTAL DE ESTABELECIMENTOS EM GESTÃO ESTADUAL ==================> {quant_cnes_estadual}", file=arquivo)

print(f" VALOR TOTAL ALOCADO NA PLANILHA ===============================> {valor_total}", file=arquivo)
print(f" VALOR TOTAL ALOCADO NA PORTARIA 90 ============================> {locale.currency(valor_portaria, grouping=True)}", file=arquivo)

print(f"\n \n====================================================== VERSÃO 2.7.10 ==================================================", file=arquivo)

# Tempo de execução
print(f" [TEMPO] - Total de execução: ===============================================================> {minutos} minutos e {segundos} segundos", file=arquivo)
print(f" [DATA HORA] - Data e hora de execução: ============================================================>", data_hora_atual.strftime("%d/%m/%Y %H:%M"), file=arquivo)

# Fechar arquivo txt
arquivo.close()

In [ ]:
arquivo.close()